# Amazon SageMaker - Bring Your Own Model 
## TensorFlow + Keras 編

ここでは TensorFlow と Keras を使ったサンプルコードを題材に、Amazon SageMaker 移行の方法を順を追って説明します。SageMaker Python SDK で TensorFlow を使うための説明は [SDK のドキュメント](https://sagemaker.readthedocs.io/en/stable/using_tf.html) にも多くの情報があります。

注: 
ここで説明するのは Script モード という記法 (現時点では標準の書き方) で、FILE モード (入力データを Amazon S3 から学習時にファイルとしてコピーする方法) です。データサイズが大きくなった場合は、FILE Mode ではなく PIPE Mode をお使い頂いた方がスループットが向上します。
また、ここでは以降手順の紹介のためトレーニングスクリプトは最小限の書き換えとしています。

## 1. トレーニングスクリプトの書き換え

### 書き換えが必要な理由
Amazon SageMaker では、オブジェクトストレージ Amazon S3 をデータ保管に利用します。例えば、S3 上の学習データを指定すると、自動的に Amazon SageMaker の学習用インスタンスにデータがダウンロードされ、トレーニングスクリプトが実行されます。トレーニングスクリプトを実行した後に、指定したディレクトリにモデルを保存すると、自動的にモデルがS3にアップロードされます。

トレーニングスクリプトを SageMaker に持ち込む場合は、以下の点を修正する必要があります。

- 学習用インスタンスにダウンロードされた学習データのロード
- 学習が完了したときのモデルの保存

これらの修正は、トレーニングスクリプトを任意の環境に持ち込む際の修正と変わらないでしょう。例えば、自身のPCに持ち込む場合も、/home/user/data のようなディレクトリからデータを読み込んで、/home/user/model にモデルを保存したいと考えるかもしれません。同様のことを SageMaker で行う必要があります。

### 書き換える前に保存先を決める
このハンズオンでは、S3からダウンロードする学習データ・バリデーションデータと、S3にアップロードするモデルは、それぞれ以下のように学習用インスタンスに保存することにします。/opt/ml/input/data/train/といったパスに設定することは奇異に感じられるかもしれませんが、これらは環境変数から読み込んで使用することが可能なパスで、コーディングをシンプルにすることができます。1-1. 環境変数の取得で読み込み方法を説明します。

#### 学習データ
- 画像: /opt/ml/input/data/train/train.npz

#### バリデーションデータ
- 画像: /opt/ml/input/data/test/test.npz

#### モデル
- /opt/ml/model 以下にシンボルやパラメータを保存する

### 書き換える箇所
まず [サンプルのソースコード](https://github.com/keras-team/keras/blob/master/examples/mnist_mlp.py) を以下のコマンドでダウンロードします。

In [ ]:
!wget https://raw.githubusercontent.com/keras-team/keras/master/examples/mnist_cnn.py

ダウンロードされた mnist_cnn.py をファイルブラウザから見つけて開いて下さい (JupyterLab の場合は左右にファイルを並べると作業しやすいです)。あるいはお好きなエディターをお使い頂いても結構です。この mnist_cnn.py は、トレーニングスクリプト内で以下の関数を呼び出し、S3以外からデータをダウンロードしていますが、SageMaker では学習データを S3 からダウンロードして使用します。

```(x_train, y_train), (x_test, y_test) = mnist.load_data()```

学習データをダウンロードして、前述したように/opt/ml/input/data/train/といったパスから読み出して使います。書き換える点は主に3点です:

1. 環境変数の取得<br>
SageMaker では、学習データやモデルの保存先はデフォルトで指定されたパスがあり、これらを環境変数から読み込んで使用することが可能です。環境変数を読み込むことで、学習データの位置をトレーニングスクリプト内にハードコーディングする必要がありません。もちろんパスの変更は可能で、API経由で渡すこともできます。

1. 学習データのロード<br>
環境変数を取得して学習データの保存先がわかれば、その保存先から学習データをロードするようにコードを書き換えましょう。

1. 学習済みモデルの保存形式と出力先の変更<br>
SageMaker では Chainer の Estimator に対して deploy 関数を呼び出すことによってモデルをデプロイします。もとのtrain_mnist.pyでは、デプロイに十分な情報がありません。このサンプルでは npz 形式を使用するため、npz 形式でモデルが保存されるようにコードを追加します。その際、モデルの保存先を正しく指定する必要があります。学習が完了すると学習用インスタンスは削除されますので、保存先を指定のディレクトリに変更して、モデルがS3にアップロードされるようにします。

### 1-0. 関数の定義
ソースコードを SageMaker で使える状態にする前に、関数を定義して処理を関数の中に入れます。

学習からモデルの保存までを train(args) 関数として定義します。ここでは次の手順で読み込む args でパラメータを受け取ります。S3 から取得したデータを読み込みトレーニングを行います。最後に、後の手順で定義する save(model, model_dir) 関数でモデルを保存します。

def train(args) を定義し、mnist_cnn.py の 16行目から最後までを、def train(args)の中に入れます。

```python
def train(args):

    batch_size = 128
    num_classes = 10
    epochs = 20

    # the data, split between train and test sets
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    (中略)

    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

```

### 1-1. 環境変数の取得
Amazon SageMaker では、トレーニングに用いるコードが実行時に Python スクリプトとして実行されます。その際、データ・モデルの入出力は こちら に記述があるよう SM_CHANNEL_XXXX や SM_MODEL_DIR という環境変数を参照する必要があります。そのため、argparse.ArgumentParser で渡された環境変数と、スクリプト実行時のハイパーパラメータを取得します。

前の手順で作成した train() 関数の下に、以下の記述を追加します。

```python
if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script
    parser.add_argument('--batch-size', type=int, default=128)
    parser.add_argument('--num-classes', type=int, default=10)
    parser.add_argument('--epochs', type=int, default=12)
    
    # input data and model directories
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--train', type=str, default=os.environ['SM_CHANNEL_TRAINING'])
    
    args, _ = parser.parse_known_args()
    train(args)
```

mnist_cnn.py の 一番上に、以下の記述を追加します。

```python
import argparse, os
```

1-2. 学習データのロード

前の手順で取得した環境変数の値を使って、train() 関数のはじめにある記述をを以下のように書き換えます。

```python
#     batch_size = 128
#     num_classes = 10
#     epochs = 12
    
    batch_size = args.batch_size
    epochs = args.epochs
    num_classes = args.num_classes
    train_dir = args.train

    # load data 
    x_train = np.load(os.path.join(train_dir, 'train.npz'))['image']
    y_train = np.load(os.path.join(train_dir, 'train.npz'))['label']
    x_test = np.load(os.path.join(train_dir, 'test.npz'))['image']
    y_test = np.load(os.path.join(train_dir, 'test.npz'))['label']
    
    # input image dimensions
    img_rows, img_cols = 28, 28

    # the data, split between train and test sets
#     (x_train, y_train), (x_test, y_test) = mnist.load_data()

```

### 1-3. 学習済みモデルの保存形式と出力先の変更

train() 関数の一番下に、以下の記述を追加します。
```python
save(model, args.model_dir)
```

mnist_cnn.py の 一番上に、以下の記述を追加します。

```python
import tensorflow as tf
from keras import backend as K
import numpy as np
```

train() 関数の下に、以下の関数を追加します。

```python
def save(model, model_dir):
    sess = K.get_session()
    tf.saved_model.simple_save(
        sess,
        os.path.join(model_dir, 'model/1'),
        inputs={'inputs': model.input},
        outputs={t.name: t for t in model.outputs})
```

https://aws.amazon.com/jp/blogs/news/amazon-sagemaker-keras/

|  S3 location  |  環境変数  |  値  |
| :---- | :---- | :----| 
|  s3://bucket_name/prefix/training  |  `SM_CHANNEL_TRAINING`  | `/opt/ml/input/data/training`  |
|  s3://bucket_name/prefix/model.tar.gz  |  `SM_MODEL_DIR`  |  `/opt/ml/model`  |

## 2. Notebook 上でのデータ準備

トレーニングを始める前に、予め Amazon S3 にデータを準備しておく必要があります。この Notebook を使ってその作業をします。

In [ ]:
import os
import keras
import numpy as np
from keras.datasets import mnist

import sagemaker

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

os.makedirs("./data", exist_ok = True)

np.savez('./data/train', image=x_train, label=y_train)
np.savez('./data/test', image=x_test, label=y_test)

In [ ]:
sagemaker_session = sagemaker.Session()
bucket_name = sagemaker_session.default_bucket()

input_data = sagemaker_session.upload_data(path='./data', bucket=bucket_name, key_prefix='data/handson-byom-tensorflow-keras')

print('Input data is uploaded to: {}'.format(input_data))

## 3. Local Mode によるトレーニングとコードの検証
トレーニングジョブを始める前に、Local Mode を使って、この Notebook インスタンス上でコンテナを立てコードをデバッグしましょう。

`from sagemaker.tensorflow import TensorFlow` で読み込んだ SageMaker Python SDK の TensorFlow Estimator を作ります。

In [ ]:
from sagemaker.tensorflow import TensorFlow
from sagemaker import get_execution_role

train_instance_type = "local"

role = get_execution_role()
estimator = TensorFlow(entry_point = "./mnist_cnn.py",
                       role=role,
                       train_instance_count=1,
                       train_instance_type=train_instance_type,
                       framework_version="1.12.0",
                       py_version='py3',
                       script_mode=True,
                       hyperparameters={'batch-size': 64,
                                        'num-classes': 10,
                                        'epochs': 1})

estimator.fit(input_data)

`estimator.fit` によりトレーニングを開始しますが、ここで「チャネル」を指定すると、環境変数名 `SM_CHANNEL_XXXX` が決定されます。この例のように何も指定しない場合、デフォルトの `SM_CHANNEL_TRAINING` となります。

`mnist.py` の中で書き換えが適切でない部分があったら、ここでエラーとなる場合があります。Local Mode ではクイックにデバッグができるので、正しく実行できるよう試行錯誤しましょう。

 `===== Job Complete =====`
と表示されれば成功です。

### 学習済みモデルの確認

In [ ]:
!aws s3 cp $estimator.model_data ./
!tar zxvf model.tar.gz

TensorFlow `SavedModel` 形式で保存されたモデルを読み込みます。

In [ ]:
import tensorflow as tf
with tf.Session() as sess:
    tf.saved_model.load(sess, [tf.saved_model.tag_constants.SERVING], "model/1/")
    
    i = sess.graph.get_tensor_by_name('conv2d_1_input:0')
    o = sess.graph.get_tensor_by_name('dense_2/Softmax:0')
    
    pred = sess.run(o, feed_dict={i:x_test[:10].reshape(-1, 28, 28, 1)})
    print('pred:', np.argmax(pred, axis=1))
    
print('true:', y_test[:10])

## 4. トレーニングジョブの発行

In [ ]:
train_instance_type = "ml.m4.xlarge"

role = get_execution_role()
estimator = TensorFlow(entry_point = "./mnist_cnn.py",
                       role=role,
                       train_instance_count=1,
                       train_instance_type=train_instance_type,
                       framework_version="1.12.0",
                       py_version='py3',
                       script_mode=True,
                       hyperparameters={'batch-size': 64,
                                        'num-classes': 10,
                                        'epochs': 4})

estimator.fit(input_data)

----
```
Billable seconds: <time>
```
と出力されればトレーニング終了です。これが実際にトレーニングインスタンスが課金される時間となります。

## 5. 推論エンドポイントのデプロイ

`estimator.deploy` で、今トレーニングしたモデルを推論エンドポイントとしてデプロイすることができます。これには数分かかります。(`----!` と表示されればデプロイ完了です。)

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

In [ ]:
import random
import matplotlib.pyplot as plt

num_samples = 5
indices = random.sample(range(x_test.shape[0] - 1), num_samples)
images, labels = x_test[indices]/255, y_test[indices]

for i in range(num_samples):
    plt.subplot(1,num_samples,i+1)
    plt.imshow(images[i].reshape(28, 28), cmap='gray')
    plt.title(labels[i])
    plt.axis('off')

In [ ]:
prediction = predictor.predict(images.reshape(-1, 28, 28, 1))['predictions']
prediction = np.array(prediction)
predicted_label = prediction.argmax(axis=1)
print('The predicted labels are: {}'.format(predicted_label))
print('true:', labels)

推論エンドポイントは立てっぱなしにしていると時間で課金されるので、確認が終わったら忘れないうちに削除してください。

In [ ]:
predictor.delete_endpoint()

## 6. Spot training
Amazon SageMaker では、[spot instance を使ったモデルの学習](https://aws.amazon.com/jp/blogs/news/managed-spot-training-save-up-to-90-on-your-amazon-sagemaker-training-jobs/) が可能です。これにより、学習にかかるコストを最大90％抑えることが可能です。

学習ジョブが終了すると、ログの最後に `Managed Spot Training savings: 64.4%` のような表示されます。この例の場合、学習にかかるコストが64.4% 削減できたことを示します。

In [ ]:
!pip install -U --quiet "sagemaker>=1.37.2"

In [ ]:

estimator = TensorFlow(entry_point = "./mnist_cnn.py",
                       role=role,
                       train_instance_count=1,
                       train_instance_type=train_instance_type,
                       framework_version="1.12.0",
                       py_version='py3',
                       script_mode=True,
                       train_max_run = 5000,
                       train_use_spot_instances = 'True',
                       train_max_wait = 10000,
                       hyperparameters={'batch-size': 64,
                                        'num-classes': 10,
                                        'epochs': 4})

estimator.fit(input_data)

## 7. まとめ

TensorFlow と Keras を使った Amazon SageMaker への移行手順について紹介しました。普段お使いのモデルでも同様の手順で移行が可能ですのでぜひ試してみてください。